### Create a training pipeline [no image preprocessing]

In [1]:
## import libraries 

import numpy as np 
import pandas as pd
from pydicom import dcmread
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch

from CT_Seg_Network.utils.dataset import *
from CT_Seg_Network.models.Unet import UNet
from CT_Seg_Network.utils.iou import IoU
from CT_Seg_Network.utils.Loss import *
from CT_Seg_Network.models.ResUnet import ResUNet
from tqdm import tqdm

In [2]:
## prepare the dataset by dividing the dataset into train and validation

IMAGE_FOLDER = 'coding_test_files/dicom_series/'
SEG_FOLDER ='coding_test_files/segmentation_data/'
patients = os.listdir(IMAGE_FOLDER)
patients.sort()

## split percentage
percentage = .8
train, val = np.split(patients, [int(len(patients)*percentage)])

def load_list(folder, list_patients):
    folder_list = []
    for i in list_patients:
        for j in os.listdir(folder + i):
            folder_list.append(folder + i + '/' + j)
    return folder_list

train_list_im = load_list(IMAGE_FOLDER, train)
val_list_im = load_list(IMAGE_FOLDER, val)

In [3]:
## create a dataset loader that will handle loading the images from dicom files directly
## and labels from segmentation data

class CSTO_CTDataset(Dataset):
    def __init__(self, image_addr_list, seg_folder=None, transforms=None):

        ## takes the address of the images
        self.images = image_addr_list
        if seg_folder == None:
            self.seg_folder = 'coding_test_files/segmentation_data/'
        else:
            self.seg_folder = seg_folder

        self.transform = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_addr = self.images[idx]

        img = dcmread(image_addr).pixel_array
        img = np.expand_dims(img, axis=0).astype(np.float32)
        img = torch.from_numpy(img).float()

        ## extract the sample name from the image_name
        last_slash = image_addr.find('dicom_series/') + 13
        sample = image_addr[last_slash:-4]
        label_addr = self.seg_folder + sample +'.npz'
        
        if os.path.isfile(label_addr):
            label = torch.from_numpy(np.transpose(np.load(label_addr)['arr_0'], (-1, 0, 1)))#.to(torch.float32)
        else:
            label = torch.zeros([4, img.shape[1], img.shape[2]])#.to(torch.float32)
    
        
        if self.transform:
            img, label = self.transform((img, label))

        sample = {'img': img,
                  'label': label}
        
        return sample
    

In [4]:
## load custom transforms that manipulates both image and label

transform_train = transforms.Compose([
            RandomFlip(),
            Resize(128),
            Normalize(mean=[0.5],
                      std=[0.5])
        ])

transform_val = transforms.Compose([
            Resize(128),
            Normalize(mean=[0.5],
                      std=[0.5])
        ])


## create sets for train and validation
trainset = CSTO_CTDataset(train_list_im, transforms=transform_train)
valset = CSTO_CTDataset(val_list_im, transforms=transform_val)

## create the dataloader for train and validation
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=16)

In [5]:
## create functions for the training and evaluation proper

def train(model,train_loader,optimizer,LOSS_FUNC,EPOCH,PRINT_INTERVAL, epoch, device):
    losses = []
    for i, batch in enumerate(tqdm(train_loader)):
        img, label = batch['img'].to(device), batch['label'].to(device)
        output = model(img)
        optimizer.zero_grad()

        loss = LOSS_FUNC(output, label)

        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        if (i + 1) % PRINT_INTERVAL == 0:
            tqdm.write('Epoch [%d/%d], Iter [%d/%d], Loss: %.4f'
                       % (epoch + 1, EPOCH, i + 1, len(train_loader), loss.item()))
    return np.mean(losses)

def dice_coefficient(y_pred, y_true):
    smooth = 1.0  # Prevent division by zero
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

def eval(model,val_loader,LOSS_FUNC, device):
    losses, dice = [], []
    for i, batch in enumerate(val_loader):
        img, label = batch['img'].to(device), batch['label'].to(device)
        output = model(img)
        loss = LOSS_FUNC(output, label)
        dice_coeff = dice_coefficient(output, label)

        dice.append(dice_coeff.item())        
        losses.append(loss.item())
    return np.mean(losses), np.mean(dice)

In [6]:
print("Train set {}\nValidation set {}".format(len(trainset),len(valset)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## initialize model type
model = ResUNet(out_classes=4).to(device)


if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).to(device)

## init optimizer, scheduler, and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
lr_sheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
LOSS_FUNC = DiceCELoss().to(device)
PRINT_INTERVAL = 5
EPOCH= 20

## start training
val_loss_epoch = []
for epoch in range(EPOCH):

    model.train()
    train_loss = train(model, trainloader, optimizer, LOSS_FUNC, EPOCH, PRINT_INTERVAL, epoch, device)
    val_loss, dice_coeff = eval(model, valloader, LOSS_FUNC, device)
    val_loss_epoch.append(val_loss)
    lr_sheduler.step()
    tqdm.write('Epoch [%d/%d], Average Train Loss: %.4f, Average Validation Loss: %.4f, Average Validation Dice Coeff.: %.4f'
                % (epoch + 1, EPOCH, train_loss, val_loss, dice_coeff))


    ## save the model with the best val result
    if val_loss == np.min(val_loss_epoch):
        print('Model saved')
        state = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        checkpoint_addr = 'CT_Seg_Network/weights/'
        torch.save(state, os.path.join(checkpoint_addr,'best.pth'))

Train set 3300
Validation set 1103


 10%|▉         | 5/52 [00:15<02:13,  2.84s/it]

Epoch [1/20], Iter [5/52], Loss: 1.0647


 19%|█▉        | 10/52 [00:29<01:52,  2.68s/it]

Epoch [1/20], Iter [10/52], Loss: 1.0277


...

Epoch [20/20], Iter [45/52], Loss: 0.9010


 96%|█████████▌| 50/52 [01:57<00:04,  2.43s/it]

Epoch [20/20], Iter [50/52], Loss: 0.8805


100%|██████████| 52/52 [02:01<00:00,  2.34s/it]


Epoch [20/20], Average Train Loss: 0.9009, Average Validation Loss: 1.0127, Average Validation Dice Coeff.: -0.4165
